The Embedding mode E5 small are get from

@article{wang2022text,
  title={Text Embeddings by Weakly-Supervised Contrastive Pre-training},
  author={Wang, Liang and Yang, Nan and Huang, Xiaolong and Jiao, Binxing and Yang, Linjun and Jiang, Daxin and Majumder, Rangan and Wei, Furu},
  journal={arXiv preprint arXiv:2212.03533},
  year={2022}
}

In [ ]:
import time
time_start = time.time()

# You need to install 'datasets' to get the data from the Hugging Face

In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 20.7 MB/s eta 0:00:00


# You need to install 'transformers' to get the Embedding mode from the Hugging Face

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import torch.utils.data as data
from transformers import AutoTokenizer, AutoModel
import torch
from torch.utils.data import DataLoader
import torch.optim as optim

In [ ]:
raw_dataset = load_dataset(path='dair-ai/emotion')
print(raw_dataset)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/dair-ai___emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


# Dataset

In [ ]:
class MyDataset(data.Dataset):
  def __init__(self, data_type):
    self.data = self.load_data(data_type)

  def load_data(self, data_type):
    tmp_data = load_dataset(path='dair-ai/emotion', split = data_type)
    Data = {}
    for idx,line in enumerate(tmp_data):
      sample = line
      Data[idx] = sample
    return Data

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

train_data = MyDataset('train')
print(len(train_data))
val_data = MyDataset('validation')
print(len(val_data))
test_data = MyDataset('test')
print(len(test_data))


16000


2000


2000


In [ ]:
print(train_data[0])

{'text': 'i didnt feel humiliated', 'label': 0}


# DataLoader

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-small')

def collote_fn(batch_samples):
  batch_text = []
  batch_label = []
  for sample in batch_samples:
    batch_text.append(sample['text'])
    batch_label.append(int(sample['label']))
  X = tokenizer(
      batch_text,
      padding=True,
      truncation=True,
      return_tensors='pt'
  )
  y = torch.tensor(batch_label)
  return X,y

train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True, collate_fn=collote_fn)
val_dataloader = DataLoader(val_data, batch_size=64, shuffle=True, collate_fn=collote_fn)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True, collate_fn=collote_fn)


In [ ]:
from torch import nn

class RNN(nn.Module):
  def __init__(self, hidden_size, output_size):
    super(RNN, self).__init__()
    self.bert_encoder = AutoModel.from_pretrained('intfloat/e5-small')
    self.hidden_size = hidden_size
    self.rnn = nn.RNN(self.bert_encoder.config.hidden_size, self.hidden_size, batch_first=True)
    self.fc = nn.Linear(self.hidden_size, output_size)
  def forward(self, x):
    bert_outputs = self.bert_encoder(**x)
    last_hidden_states = bert_outputs.last_hidden_state
    # x的形状为(batch_size, seq_len, input_size)
    outputs, hidden = self.rnn(last_hidden_states)
    # 取最后一个时间步的输出作为分类器的输入
    out = self.fc(outputs[:, -1, :])
    return out

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RNN(256,6).to(device)
print(model)

RNN(
  (bert_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affin

In [ ]:
from tqdm.auto import tqdm

def train(dataloader, model, loss_fun, optimizer, epoch, total_loss):
  process_bar = tqdm(range(len(dataloader)))
  process_bar.set_description(f'loss: {0:>7f}')
  finish_batch_num = (epoch-1)*len(dataloader)

  model.train()
  for batch, (X,y) in enumerate(dataloader, start=1):
    X,y = X.to(device),y.to(device)
    pred = model(X)
    loss = loss_fun(pred,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    process_bar.set_description(f'loss: {total_loss/(finish_batch_num + batch):>7f}')
    process_bar.update(1)
  return total_loss


def test(dataloader, model, mode='Test'):
    assert mode in ['Valid', 'Test']
    target_num = torch.zeros((1, 6)) # n_classes为分类任务类别数量
    predict_num = torch.zeros((1, 6))
    acc_num = torch.zeros((1, 6))
    total_val_loss = 0
    avg_val_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            loss = loss_fun(outputs,y)
            total_val_loss += loss.item()
            pred = outputs.argmax(1)
            # correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            pre_mask = torch.zeros(outputs.size()).scatter_(1, pred.cpu().view(-1, 1), 1.)
            predict_num += pre_mask.sum(0)
            tar_mask = torch.zeros(outputs.size()).scatter_(1, y.data.cpu().view(-1, 1), 1.)
            target_num += tar_mask.sum(0)
            acc_mask = pre_mask * tar_mask 
            acc_num += acc_mask.sum(0)    
        # calculate accuracy
        avg_val_loss = total_val_loss / len(dataloader)
        recall = torch.nan_to_num(acc_num / target_num, nan=0.0)
        precision = torch.nan_to_num(acc_num / predict_num, nan=0.0)
        F1 = torch.nan_to_num(2 * recall * precision / (recall + precision), nan=0.0)
        accuracy = 100. * acc_num.sum(1) / target_num.sum(1)

        avg_precision = torch.mean(precision)
        avg_recall = torch.mean(recall)
        avg_f1 = torch.mean(F1)

        print('{},loss{}, Acc {}, recal {}, precision {}, F1-score {}'.format(mode, avg_val_loss,accuracy.tolist(), avg_recall.tolist(), avg_precision.tolist(), avg_f1.tolist()))     
    return avg_val_loss,accuracy,avg_precision,avg_recall,avg_f1

In [ ]:
learn_rate = 1e-5
num_epochs = 10

loss_fun = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learn_rate)

total_loss = 0
best_acc = 0

# 将结果写入CSV文件中,训练和验证
import csv
with open('result.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Epoch','vail_loss','vail_accuracy','vail_precision', 'vail_recall', 'vail_F1-score'])

    for t in range(num_epochs):
      total_loss = train(train_dataloader, model, loss_fun, optimizer, t+1, total_loss)
      avg_val_loss,vaild_acc,vaild_pre,vaild_recall,vaild_f1 = test(val_dataloader,model,mode='Valid')
      writer.writerow([t+1,avg_val_loss,vaild_acc,vaild_pre, vaild_recall, vaild_f1])
      if vaild_acc > best_acc:
        best_acc = vaild_acc
        torch.save(model.state_dict(), 'best_model.pt')


  0%|          | 0/250 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Valid,loss0.8776917103677988, Acc [70.9000015258789], recal 0.4912544786930084, precision 0.46082791686058044, F1-score 0.4703863561153412


  0%|          | 0/250 [00:00<?, ?it/s]

Valid,loss0.49212948326021433, Acc [84.80000305175781], recal 0.7482655644416809, precision 0.8396902680397034, F1-score 0.7690737843513489


...

Valid,loss0.1761835202923976, Acc [92.9000015258789], recal 0.9063536524772644, precision 0.8959242701530457, F1-score 0.9005846977233887


  0%|          | 0/250 [00:00<?, ?it/s]

Valid,loss0.19119506026618183, Acc [92.55000305175781], recal 0.9139658808708191, precision 0.8879734873771667, F1-score 0.8996513485908508


  0%|          | 0/250 [00:00<?, ?it/s]

Valid,loss0.16632893215864897, Acc [93.25], recal 0.9217343926429749, precision 0.8985588550567627, F1-score 0.90889573097229


In [ ]:
time_end = time.time()  # 记录结束时间
time_sum = time_end - time_start  # 计算的时间差为程序的执行时间，单位为秒/s
print(f"Run time: {time_sum:.2f}")

Run time: 667.43


#test,加载模型,进行预测

In [ ]:
model.load_state_dict(torch.load('/content/best_model.pt'))
avg_test_loss,test_acc,test_pre,test_recall,test_f1 = test(test_dataloader,model,mode='Test')

Test,loss0.16515745338983834, Acc [92.5], recal 0.9059498906135559, precision 0.8703052997589111, F1-score 0.8850038051605225


In [ ]:
print('Device name:', torch.cuda.get_device_name(0))

Device name: Tesla T4


In [ ]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement()

print('trainable parameters: {:e}'.format(param_size))

trainable parameters: 3.352589e+07
